In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [81]:
#read_files
match=pd.read_csv('matches.csv')
delivery=pd.read_csv('deliveries.csv')

In [82]:
#rename id cols to get same name for both tables
match.rename(columns={'id': 'match_id'},inplace=True)

In [83]:
#calculate total runs for match and particular innings
tot_score_df=delivery.groupby(['match_id','inning'])['total_runs'].sum().reset_index()

In [84]:
#separate total runs (target) set by first team
tot_score_df = tot_score_df[tot_score_df['inning']==1]

In [85]:
#merge the total score and match details table
match_df=tot_score_df.merge(match,on = 'match_id')

In [86]:
#remove innings colum
match_df.drop('inning',axis=1,inplace=True)

In [87]:
match_df.head()

,match_id,total_runs,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,207,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,184,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,183,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,163,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,157,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [88]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [89]:
teams = [
    'Sunrisers Hyderabad', 
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',  
    'Kings XI Punjab',
    'Chennai Super Kings', 
    'Rajasthan Royals', 
    'Delhi Capitals'
]

In [90]:
#rename certain team names
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [91]:
match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]

In [93]:
#exclude rain affected matches
match_df=match_df[match_df['dl_applied']==0]

In [94]:
match_df.columns

Index(['match_id', 'total_runs', 'Season', 'city', 'date', 'team1', 'team2',
       'toss_winner', 'toss_decision', 'result', 'dl_applied', 'winner',
       'win_by_runs', 'win_by_wickets', 'player_of_match', 'venue', 'umpire1',
       'umpire2', 'umpire3'],
      dtype='object')

In [97]:
#extract relevant features from match dataset and merge with delivery dataset
features=['match_id', 'total_runs','city','winner']

In [98]:
match_df=match_df[features]

In [102]:
delivery_df=match_df.merge(delivery, on ='match_id')

In [104]:
#only focus on 2nd innings in delivery dataset to generate win probablility
delivery_df=delivery_df[delivery_df['inning']==2]

In [105]:
delivery_df

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder
125,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
127,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
128,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,0,0,2,0,2,NaN,NaN,NaN
129,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,0,0,4,0,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149573,11415,152,Hyderabad,Mumbai Indians,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,0,0,0,1,0,1,NaN,NaN,NaN
149574,11415,152,Hyderabad,Mumbai Indians,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,0,0,0,2,0,2,NaN,NaN,NaN
149575,11415,152,Hyderabad,Mumbai Indians,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,0,0,0,0,1,0,1,SR Watson,run out,KH Pandya
149576,11415,152,Hyderabad,Mumbai Indians,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,0,0,0,2,0,2,NaN,NaN,NaN


In [137]:
#calculate balls left data
delivery_df['balls_left'] = 120-((delivery_df['over'] - 1)*6 + delivery_df['ball']) 

In [129]:
#calculate total score ball by ball
delivery_df['score_by_ball'] = delivery_df.groupby('match_id')['total_runs_y'].cumsum()

In [134]:
#calculate run needed after each ball
delivery_df['run_needed'] = delivery_df['total_runs_x'] - delivery_df['score_by_ball'] 

In [148]:
#calculate current run rate and required run rate
delivery_df['crr'] = (delivery_df['score_by_ball'] / ((delivery_df['over'] - 1)*6 + delivery_df['ball']))*6 
delivery_df['rrr'] = (delivery_df['run_needed'] / delivery_df['balls_left'])*6 

In [149]:
delivery_df.head()

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,batsman,...,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,score_by_ball,run_needed,balls_left,crr,rrr
125,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,1,NaN,NaN,NaN,1,206,119,6.0,10.386555
126,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,NaN,NaN,NaN,1,206,118,3.0,10.474576
127,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,NaN,NaN,NaN,1,206,117,2.0,10.564103
128,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,2,NaN,NaN,NaN,3,204,116,4.5,10.551724
129,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,4,NaN,NaN,NaN,7,200,115,8.4,10.434783


In [165]:
#generate wickets remaining data
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x : x if x == "0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
delivery_df['player_dismissed'] = delivery_df.groupby('match_id')['player_dismissed'].cumsum()
delivery_df['wickets_remaining'] = 10 - delivery_df['player_dismissed']        

In [186]:
#generate new col- result basis on whether 2nd batting team was the winnerdef result(row):
    return 1 if row['batting_team']==row['winner'] else 0

In [187]:
delivery_df['result'] = delivery_df.apply(result, axis=1)

In [189]:
delivery_df.head()

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,batsman,...,player_dismissed,dismissal_kind,fielder,score_by_ball,run_needed,balls_left,crr,rrr,wickets_remaining,result
125,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,NaN,NaN,1,206,119,6.0,10.386555,10,0
126,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,NaN,NaN,1,206,118,3.0,10.474576,10,0
127,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,NaN,NaN,1,206,117,2.0,10.564103,10,0
128,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,NaN,NaN,3,204,116,4.5,10.551724,10,0
129,1,207,Hyderabad,Sunrisers Hyderabad,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,NaN,NaN,7,200,115,8.4,10.434783,10,0


In [194]:
#select final dataset ; exclude relevant cols
final_df=delivery_df[['batting_team','bowling_team','city','run_needed','balls_left','wickets_remaining','crr','rrr','total_runs_x','result']]

In [211]:
#identify NAs and drop rows with NAs
final_df.isna().sum()

batting_team           0
bowling_team           0
city                 832
run_needed             0
balls_left             0
wickets_remaining      0
crr                    0
rrr                    7
total_runs_x           0
result                 0
dtype: int64

In [212]:
final_df.dropna(inplace=True)

In [197]:
#shuffle the data to eliminate ball by ball bias of a same match
final_df=final_df.sample(final_df.shape[0])

In [219]:
#remove rows where balls_left =0 (Excluding crr, nrr = infinity situations)
final_df= final_df[final_df['balls_left']!=0]

In [227]:
#import ML libraries
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [220]:
#create test train split
x = final_df[['batting_team','bowling_team','city','run_needed','balls_left','wickets_remaining','crr','rrr','total_runs_x']]
y=final_df['result']
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2, random_state=1)

In [230]:
#implement column transformer and het OHE for categorical columns
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse = False, drop = 'first'), ['batting_team','bowling_team','city'])
], remainder = 'passthrough') 

In [249]:
#create pipeline structure - using Logistic Regression
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [223]:
xtrain.describe()

,run_needed,balls_left,wickets_remaining,crr,rrr,total_runs_x
count,57073.000000,57073.000000,57073.000000,57073.000000,57073.000000,57073.000000
mean,92.585531,62.837822,7.550050,7.446296,10.391414,165.714821
std,49.896054,33.249373,2.127133,2.282466,13.793803,29.282081
min,-12.000000,-2.000000,0.000000,0.000000,-510.000000,65.000000
25%,53.000000,35.000000,6.000000,6.272727,7.148936,147.000000
50%,92.000000,63.000000,8.000000,7.485149,8.883117,165.000000
75%,130.000000,92.000000,9.000000,8.692308,10.909091,185.000000
max,248.000000,119.000000,10.000000,42.000000,678.000000,250.000000


In [250]:
#train the model
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [251]:
#predict test values
ypred= pipe.predict(xtest)

In [252]:
#get accuracy scores
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.8018781974910646

In [261]:
pipe.predict_proba(xtest)[2]

array([0.90464424, 0.09535576])

In [241]:
#in the 2nd model use Random forest instead of Logistic Regression
from sklearn.ensemble import RandomForestClassifier

In [243]:
pipe2=Pipeline(steps=[
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

In [244]:
pipe2.fit(xtrain,ytrain)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', RandomForestClassifier())])

In [246]:
ypred2= pipe2.predict(xtest)

In [247]:
accuracy_score(ytest,ypred2)

0.9988086060691008